In [2]:
import pandas as pd
from Bio.Blast import NCBIWWW
from Bio.Blast.Applications import NcbiblastxCommandline

In [3]:
import pandas as pd

bovine_ms_aa= pd.read_csv("MS_Data_with_percent_homology.csv")
bovine_ms_aa.head(10)

,Unnamed: 0,Unique,Total,reference,Gene Symbol,MWT(kDa),AVG,Bovine_Seq,%identity_with_chlammy
0,0,133,154,tr|A0A3Q1NE82|A0A3Q1NE82_BOVIN,AHNAK,614.26,3.1225,MEKEEETTRELLLPNWQGSGSHGLTITQRDDGVFVQEVMQNSPAAR...,0.000
1,1,7,109,sp|Q3MHM5|TBB4B_BOVIN,TUBB4B,49.80,3.3008,MREIVHLQAGQCGNQIGAKFWEVISDEHGIDPTGTYHGDSDLQLER...,90.465
2,2,13,101,sp|Q2T9S0|TBB3_BOVIN,TUBB3,50.40,3.3130,MREIVHIQAGQCGNQIGAKFWEVISDEHGIDPSGNYVGDSDLQLER...,85.581
3,3,62,98,tr|A0A3Q1M0Y8|A0A3Q1M0Y8_BOVIN,CROCC2,169.52,3.1375,MDLEDALGRLEAAEQRSTSLSQVNALLREQLEQLRKAHDRLAEELA...,0.000
4,4,16,91,tr|F6RP72|F6RP72_BOVIN,LOC100295712; tubulin alpha chain,50.85,3.0554,MPHSLLCFQRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPS...,87.097
5,5,80,88,tr|F1MMQ8|F1MMQ8_BOVIN,SPTA1,280.78,3.3721,MESDGPEVLETAEEIQARRQEVLSRYQRFKERVAERGQKLEDSYHY...,0.000
6,6,71,84,tr|F1MKE9|F1MKE9_BOVIN,SPTB,268.25,3.5225,MTSATEFENVGNQPPYSRINARWDGPDDELDNDNSSARLFERSRIK...,0.000
7,7,18,83,tr|A0A452DIL8|A0A452DIL8_BOVIN,TUBB2B,49.94,3.7333,MREIVHIQAGQCGNQIGAKFWEVISDEHGIDPTGSYHGDSDLQLER...,89.767
8,8,27,75,tr|Q58DT9|Q58DT9_BOVIN,ACTA2,45.19,2.7932,MCEEEDSTALVCDNGSGLCKAGFAGDDAPRAVFPSIVGRPRHQGVM...,87.870
9,9,68,72,tr|A0A3Q1MRS3|A0A3Q1MRS3_BOVIN,PLEC,535.22,2.9450,MVAGMFMPLDQLRAIYEVLFREGVMVAKKDRRPRSLHPHVPGVTNL...,49.533


In [95]:
chlammy_seq_match=[]

for n in range(bovine_ms_aa.shape[0]):
    with open("test.txt",'w') as f:
        f.write(bovine_ms_aa["Bovine_Seq"][n])

    blastx_cline = NcbiblastxCommandline(cmd="blastp",query="test.txt", db="Chlammy_seqs.fasta", evalue=0.001,outfmt=6)
    J=blastx_cline()
    J=J[0].split("\n")
    try:
        chlammy_seq_match.append(J[0].split("\t")[1])
    except:
        chlammy_seq_match.append(0)

In [96]:
bovine_ms_aa['ChlammySeqName']=chlammy_seq_match

Apparently when I saved the dataframe as a csv I did not use the argument to not include an index. Ill drop that here.

In [97]:
bovine_ms_aa=bovine_ms_aa.drop(["Unnamed: 0"],axis=1)

Some functions to programatically access the Uniprot database

In [5]:
import urllib.parse
import urllib.request

def req_uniport(gene):
    #Specify the database to query - we are just quering the general protein database
    url = 'https://www.uniprot.org/uniprot/'

    #entering the parameters we care about
    params= {
        'query':gene,
        'format':'fasta'
    }

    data=urllib.parse.urlencode(params)
    data = data.encode('utf-8')
    req = urllib.request.Request(url,data)
    
    return req

def decode_uniport(req):
    with urllib.request.urlopen(req) as f:
        response = f.read()
        response = response.decode('utf-8')
    
    return response

def from_response_get_aaonly(response):
    start = int(0)

    for n_line, i in enumerate(response.split()):
        if i[:2] == "SV":
            start=n_line
            break
    return "".join(response.split()[n_line+1:])

    
def get_uniprot_aa_seq(gene):
    request=req_uniport(gene)
    response=decode_uniport(request)
    seq=from_response_get_aaonly(response)
    return seq

def get_uniprot_response(gene):
    request=req_uniport(gene)
    return decode_uniport(request)
    
    

In [99]:
gene_names=[]
for k,i in enumerate(bovine_ms_aa['ChlammySeqName']):
    c=0
    if i == 0:
        gene_names.append(0)
        continue
    fasta=decode_uniport(req_uniport(i))
    for block in fasta.split():
        if "GN=" in block:
            gene_names.append(block[3:])
            c=1
            break
    
    if c==0:
        try:
            gene_names.append("_".join(fasta.split()[0].split("|")[1:3]))
        except:
            gene_names.append("Seq not in Uniprot")
        

Just want to change around the columns

In [100]:
bovine_ms_aa['Chlammy_GeneName']=gene_names

In [104]:
bovine_ms_aa.columns.tolist()

['Unique',
 'Total',
 'reference',
 'Gene Symbol',
 'MWT(kDa)',
 'AVG',
 'Bovine_Seq',
 '%identity_with_chlammy',
 'ChlammySeqName',
 'Chlammy_GeneName']

In [107]:
new_order=['Unique',
 'Total',
 'reference',
 'Gene Symbol',
 'Chlammy_GeneName',
 'MWT(kDa)',
 'AVG',
 'Bovine_Seq',
 '%identity_with_chlammy',
 'ChlammySeqName']

In [110]:
bovine_ms_aa=bovine_ms_aa[new_order]

In [111]:
bovine_ms_aa.to_csv("MS_HomologyANDChlammyName.csv",index=False)

Apparently some sequences are not reconized in the Uniprot database. This is an example where a sequence was found, but the uniprot did not have the sequence. Perhaps I could "unbreak" this by not just dumbly taking the top sequence hit from the blast?

In [82]:
bovine_ms_aa.loc[[121]]

,Unique,Total,reference,Gene Symbol,MWT(kDa),AVG,Bovine_Seq,%identity_with_chlammy,ChlammySeqName
121,9,11,sp|P68432|H31_BOVIN,Histone H3.1,15.39,2.2491,MARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRYRPGT...,95.588,S59581


In [9]:
import pandas as pd

df= pd.read_csv("MS_HomologyANDChlammyName.csv")
df.head(10)
df.columns

Index(['Unique', 'Total', 'reference', 'Gene Symbol', 'Chlammy_GeneName',
       'MWT(kDa)', 'AVG', 'Bovine_Seq', '%identity_with_chlammy',
       'ChlammySeqName'],
      dtype='object')

In [12]:
n_res=[]
for seq in df["Bovine_Seq"]:
    n_res.append(len(seq))

In [15]:
df["n_res_bov"] = n_res


In [16]:
df


,Unique,Total,reference,Gene Symbol,Chlammy_GeneName,MWT(kDa),AVG,Bovine_Seq,%identity_with_chlammy,ChlammySeqName,n_res_bov
0,133,154,tr|A0A3Q1NE82|A0A3Q1NE82_BOVIN,AHNAK,0,614.26,3.1225,MEKEEETTRELLLPNWQGSGSHGLTITQRDDGVFVQEVMQNSPAAR...,0.000,0,5768
1,7,109,sp|Q3MHM5|TBB4B_BOVIN,TUBB4B,TUBB1,49.80,3.3008,MREIVHLQAGQCGNQIGAKFWEVISDEHGIDPTGTYHGDSDLQLER...,90.465,AAA33101.1,445
2,13,101,sp|Q2T9S0|TBB3_BOVIN,TUBB3,TUBB1,50.40,3.3130,MREIVHIQAGQCGNQIGAKFWEVISDEHGIDPSGNYVGDSDLQLER...,85.581,AAA33101.1,450
3,62,98,tr|A0A3Q1M0Y8|A0A3Q1M0Y8_BOVIN,CROCC2,0,169.52,3.1375,MDLEDALGRLEAAEQRSTSLSQVNALLREQLEQLRKAHDRLAEELA...,0.000,0,1535
4,16,91,tr|F6RP72|F6RP72_BOVIN,LOC100295712; tubulin alpha chain,TUA2,50.85,3.0554,MPHSLLCFQRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPS...,87.097,AAN87017.1,458
...,...,...,...,...,...,...,...,...,...,...,...
645,1,1,sp|Q32KN5|DNAL4_BOVIN,DNAL4,MOT24,12.00,2.3611,MGETEGKKDEADYKRLQTFPLVRHSDMPEEMRVETMELCVTACEKF...,50.485,ACC68802.1,105
646,1,1,tr|A0A3Q1N924|A0A3Q1N924_BOVIN,NaN,0,43.53,2.3494,MFNLMKKDKDKDGGRKEKKEKKEKKERMSAAELRSLEEMSLRRGFF...,0.000,0,398
647,1,1,tr|A0A452DJE2|A0A452DJE2_BOVIN,PRKACB,CHLREDRAFT_121355,46.06,2.3489,MAAYREVPCNQYTGTTALQKLEGFASRLFHRHSKGTAHDQKTALEN...,44.674,XP_001699230.1,397
648,1,1,tr|A0A3Q1MM27|A0A3Q1MM27_BOVIN,GNAZ,CGL49,40.71,2.3416,MGCRQSSEEKEAARRSRRIDRHLRSESQRQRREIKLLLLGTSNSGK...,26.882,AEB71789.1,355


In [31]:
for name in df['ChlammySeqName']:
    if name == '0':
        continue
    print(name)

AAA33101.1
AAA33101.1
AAN87017.1
AAA33101.1
BAA09450.1
XP_001696699.1
AAA50455.1
PNW77020.1
PNW83408.1
XP_001691545.1
XP_001700194.1
PNW83408.1
PNW73318.1
AAN87017.1
XP_001701792.1
XP_001696752.1
AAA33083.1
XP_001695126.1
AAA33101.1
AAA33092.1
PNW83807.1
XP_001696752.1
DAA00063.1
XP_001694675.1
DAA00063.1
XP_001703170.1
ABC02020.1
XP_001697746.1
XP_001701436.1
XP_001698838.1
PNW75124.1
XP_001700526.1
XP_001692878.1
XP_001693673.1
PNW82351.1
CAB99316.1
XP_001702419.1
XP_001693839.1
PNW72966.1
PNW70875.1
PNW86922.1
XP_001695420.1
BAA09450.1
BAP87096.1
XP_001696503.1
XP_001695440.1
ABC02021.1
AAF34540.1
XP_001690436.1
XP_001690930.1
XP_001690875.1
XP_001700194.1
XP_001700741.1
PNW73098.1
XP_001697821.1
XP_001691996.1
CAA88139.1
PNW75905.1
PNW78852.1
CAA39053.1
PNW85595.1
PNW79521.1
XP_001695126.1
PNW84287.1
PNW86245.1
PNW75212.1
PNW85082.1
BAP87095.1
PNW69867.1
XP_001699797.1
CAA32685.1
XP_001701326.1
XP_001701565.1
XP_001702484.1
XP_001702841.1
AAD45352.1
AAS15573.1
CAA43808.1
PNW71649.1

In [86]:
from Bio import Entrez, SeqIO

seq_list=[]
for name in df['ChlammySeqName']:
    print(name)
    if name == '0':
        seq_list.append("0")
        continue
        
    Entrez.email = "jacob_r_anderson@hms.harvard.edu"
    handle=Entrez.efetch(db="protein",id=name,rettype="gb", retmode="text")
    records = SeqIO.parse(handle, "gb")

    
    for record in records:
        seq_string=""
        for seq in record:
            seq_string+=seq
        seq_list.append(seq_string)

            #print(line.split("\n"))


0
AAA33101.1
AAA33101.1
0
AAN87017.1
0
0
AAA33101.1
BAA09450.1
XP_001696699.1
AAA50455.1
PNW77020.1
PNW83408.1
XP_001691545.1
XP_001700194.1
PNW83408.1
PNW73318.1
AAN87017.1
0
XP_001701792.1
XP_001696752.1
AAA33083.1
XP_001695126.1
AAA33101.1
0
AAA33092.1
PNW83807.1
0
0
0
0
XP_001696752.1
0
DAA00063.1
0
0
XP_001694675.1
DAA00063.1
XP_001703170.1
0
ABC02020.1
XP_001697746.1
XP_001701436.1
XP_001698838.1
0
PNW75124.1
XP_001700526.1
XP_001692878.1
XP_001693673.1
PNW82351.1
CAB99316.1
XP_001702419.1
XP_001693839.1
PNW72966.1
0
PNW70875.1
PNW86922.1
0
XP_001695420.1
BAA09450.1
BAP87096.1
XP_001696503.1
XP_001695440.1
0
ABC02021.1
AAF34540.1
XP_001690436.1
XP_001690930.1
0
XP_001690875.1
XP_001700194.1
XP_001700741.1
PNW73098.1
0
XP_001697821.1
XP_001691996.1
CAA88139.1
PNW75905.1
0
PNW78852.1
CAA39053.1
PNW85595.1
PNW79521.1
0
0
XP_001695126.1
PNW84287.1
PNW86245.1
PNW75212.1
PNW85082.1
0
BAP87095.1
PNW69867.1
XP_001699797.1
CAA32685.1
0
XP_001701326.1
0
XP_001701565.1
XP_001702484.1
0
XP_0

In [90]:
n_res_chlam=[]
for seq in seq_list:
    if seq == "0":
        n_res_chlam.append("0")
        continue
    n_res_chlam.append(len(seq))
    
    

In [91]:
n_res_chlam

['0',
 443,
 443,
 '0',
 451,
 '0',
 '0',
 443,
 377,
 173,
 4485,
 399,
 4309,
 129,
 121,
 4309,
 4191,
 451,
 '0',
 501,
 1033,
 163,
 4568,
 443,
 '0',
 465,
 1306,
 '0',
 '0',
 '0',
 '0',
 1033,
 '0',
 635,
 '0',
 '0',
 240,
 635,
 4625,
 '0',
 269,
 4143,
 552,
 1719,
 '0',
 396,
 264,
 682,
 403,
 294,
 4513,
 4653,
 868,
 288,
 '0',
 505,
 1711,
 '0',
 110,
 377,
 904,
 817,
 1941,
 '0',
 216,
 367,
 1029,
 615,
 '0',
 346,
 121,
 3553,
 1138,
 '0',
 94,
 482,
 253,
 787,
 '0',
 4165,
 567,
 1901,
 697,
 '0',
 '0',
 4568,
 628,
 462,
 867,
 404,
 '0',
 925,
 442,
 230,
 516,
 '0',
 651,
 '0',
 434,
 270,
 '0',
 380,
 1024,
 586,
 '0',
 574,
 707,
 764,
 576,
 '0',
 '0',
 662,
 316,
 1929,
 '0',
 264,
 '0',
 682,
 763,
 '0',
 '0',
 135,
 261,
 93,
 674,
 759,
 705,
 1165,
 961,
 1074,
 '0',
 '0',
 198,
 125,
 '0',
 3124,
 698,
 412,
 223,
 663,
 705,
 '0',
 204,
 187,
 443,
 611,
 1057,
 734,
 356,
 569,
 705,
 641,
 1051,
 239,
 169,
 '0',
 230,
 261,
 318,
 105,
 86,
 129,
 19

In [92]:
df["ChlammySeq"]=seq_list
df["n_res_chlam"]=n_res_chlam

    



In [94]:
df.columns.to_list()

['Unique',
 'Total',
 'reference',
 'Gene Symbol',
 'Chlammy_GeneName',
 'MWT(kDa)',
 'AVG',
 'Bovine_Seq',
 '%identity_with_chlammy',
 'ChlammySeqName',
 'n_res_bov',
 'ChlammySeq',
 'n_res_chlam']

In [95]:
new_order_2=['Unique',
 'Total',
 'AVG',
 'MWT(kDa)',
 'reference',
 'Gene Symbol',
 'Chlammy_GeneName',
 '%identity_with_chlammy',
 'n_res_bov',
 'n_res_chlam',
 'Bovine_Seq',
 'ChlammySeq',
 'ChlammySeqName',
]

In [96]:
df=df[new_order_2]

In [99]:
df.to_csv("MS_Data_AnnotedbyBlast.csv",index=False)

In [25]:
chlammy_seq_match=[]

for n in range(1,2):#range(bovine_ms_aa.shape[0]):
    with open("test.txt",'w') as f:
        f.write(bovine_ms_aa["Bovine_Seq"][n])

    blastx_cline = NcbiblastxCommandline(cmd="blastp",query="test.txt", db="Chlammy_seqs.fasta", evalue=0.001,outfmt=7)
    J=blastx_cline()
    #J=J[0].split("\n")
    for line in J:
        print(line)
#     try:
#         chlammy_seq_match.append(J[0].split("\t")[1])
#     except:
#         chlammy_seq_match.append(0)

# BLASTP 2.10.0+
# Query: 
# Database: Chlammy_seqs.fasta
# Fields: query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score
# 30 hits found
Query_1	AAA33101.1	90.465	430	41	0	1	430	1	430	0.0	813
Query_1	AAA33102.1	90.465	430	41	0	1	430	1	430	0.0	813
Query_1	XP_001693997.1	90.465	430	41	0	1	430	1	430	0.0	813
Query_1	XP_001694072.1	90.465	430	41	0	1	430	1	430	0.0	813
Query_1	PNW76052.1	90.465	430	41	0	1	430	1	430	0.0	813
Query_1	PNW76245.1	90.465	430	41	0	1	430	1	430	0.0	813
Query_1	P04690.1	90.465	430	41	0	1	430	1	430	0.0	813
Query_1	AAN87017.1	40.553	434	248	2	1	424	1	434	3.68e-120	358
Query_1	AAA33095.1	40.553	434	248	2	1	424	1	434	3.68e-120	358
Query_1	XP_001691876.1	40.553	434	248	2	1	424	1	434	3.68e-120	358
Query_1	XP_001703110.1	40.553	434	248	2	1	424	1	434	3.68e-120	358
Query_1	PNW83770.1	40.553	434	248	2	1	424	1	434	3.68e-120	358
Query_1	PNW85531.1	40.553	434	248	2	1	424	1	434	3.68e-120	358
Query_